# Implementation of different versions of the benefit difference fairness function

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import multiprocessing as mp
from src.feature_map import IdentityFeatureMap
from src.functions import cost_utility, demographic_parity
from src.plotting import plot_results_over_lambdas
from src.training import train_multiple

## The parameters used by the the original authors  

In [ ]:
dim_x = 1
training_parameters = {
    'keep_collected_data': False,
    'use_sensitve_attributes': False,
    'time_steps':200,
    'batch_size':512,
    'num_iterations': 32,
    'learning_parameters': {
        'learning_rate': 0.5,
        'decay_rate': 0.8,
        'decay_step': 30
    },
    #'fairness_rate':1000,
    'fraction_protected':0.5,
    'num_test_samples': 2000,
    'bias': True,
    'benefit_value_function': demographic_parity
}
def util_func(**util_params):
    util = cost_utility(cost_factor=0.55, **util_params)
    return util

dim_theta = dim_x + 1 if training_parameters['bias'] else dim_x
training_parameters['theta'] = [-3.5, 0.6]
training_parameters['feature_map'] = IdentityFeatureMap(dim_theta)
training_parameters['num_decisions'] = training_parameters['num_iterations'] * training_parameters['batch_size']
training_parameters['utility_value_function'] = util_func


## Correct Benefit Difference

In [ ]:
def fairness_function(**fairness_kwargs):
    policy = fairness_kwargs["policy"]
    x = fairness_kwargs["x"]
    s = fairness_kwargs["s"]
    y = fairness_kwargs["y"]
    decisions = fairness_kwargs["decisions"]
    ips_weights = fairness_kwargs["ips_weights"]

    benefit = policy.benefit_function(decisions=decisions, y=y)
    phi = policy.feature_map(policy._extract_features(x, s))
    denominator = np.expand_dims((1.0 + np.exp(np.matmul(phi, policy.theta))), axis=1)

    benefit_grad = benefit * phi/denominator

    if ips_weights is not None:
        benefit *= ips_weights
        benefit_grad *= ips_weights
        
    # benefit-difference * grad-benefit-difference
    return policy._mean_difference(benefit, s) * policy._mean_difference(benefit_grad, s)

training_parameters['fairness_function'] = fairness_function
lambdas = np.logspace(-1, 5, base=10, endpoint=True, num=10)
lambdas = np.insert(arr=lambdas, obj=0, values=[0.0])

results = train_multiple(training_parameters, iterations=5, lambdas=lambdas, verbose=True)
plot_results_over_lambdas(
    training_parameters["utility_stats"]["mean"], 
    training_parameters["utility_stats"]["max"], 
    training_parameters["utility_stats"]["min"], 
    training_parameters["benefit_delta_stats"]["mean"], 
    training_parameters["benefit_delta_stats"]["max"], 
    training_parameters["benefit_delta_stats"]["min"], 
    lambdas, 
    file_path="results.png")